## Rumah123 Web Scraper
### Ida Bagus Raditya Avanindra Mahaputra / 18219117
### Tugas Calon Asisten Lab Basis Data

## Import all necessary libraries

In [26]:
import bs4
from urllib.request import urlopen as uReq, Request
from bs4 import BeautifulSoup as soup
import time
from tqdm import tqdm
import json

In [27]:
from geopy.geocoders import Nominatim
geocoder = Nominatim(user_agent = 'rumah123scraper')

## Create House Data Class

In [28]:
class HouseData:
    def __init__(self,property_title, property_agent,location,latitude,longitude,phone_number,property_type,land_area,building_area,price_idr,num_bathroom,num_bedroom,garage_capacity):
        self.property_title=property_title
        self.property_agent=property_agent
        self.location=location
        self.latitude=latitude
        self.longitude=longitude
        self.phone_number=phone_number
        self.property_type=property_type
        self.land_area=land_area
        self.building_area=building_area
        self.price_idr=price_idr
        self.num_bathroom=num_bathroom
        self.num_bedroom=num_bedroom
        self.garage_capacity=garage_capacity

    def printObject(self):
        print(f'''
            Title : {self.property_title}
            Agen: {self.property_agent}
            Location : {self.location}
            Latitude: {self.latitude}
            Longitude: {self.longitude}
            Phone number: {self.phone_number}
            Type : {self.property_type}
            LT : {self.land_area}
            LB : {self.building_area}
            price : {self.price_idr}
            KM: {self.num_bathroom}
            KT: {self.num_bedroom}
            Kapasitas Garasi Mobil: {self.garage_capacity}
            ''')

## Create functions for data manipulation

In [29]:
def convert_harga_to_int(str):
    # format "Rp. 700 Jt" atau "Rp. 35 M" atau "Rp. 355"
    str = str.replace(",", ".") # mengubah koma menjadi titik untuk kebutuhan lfoat
    str_split = str.split()

    if len(str_split)<=2:
        # harga hanya mengambil bentuk integer dari indeks ke 1
        harga = int(float(str_split[1])*1000000) #asumsi semua dalam satuan juta
        return harga
    else:
        # len dari listnya >=3 (ada Jt atau M)
        if str_split[2].lower() == 'jt':
            harga = int(float(str_split[1])*1000000)
            return harga
        elif str_split[2].lower() == 'm':
            harga = int(float(str_split[1])*1000000000)
            return harga

## Create function to scrape the website

This function scrapes the website and creates an object based on the values it got. While scraping, the functions also preprocess the data so no null/bad values is added to the object list

In [51]:
def scrapeweb(listOfHouse, firstpage, lastpage):
    
    i = firstpage
    while i < lastpage+1:
        try: 
            # declare the url for the website
            base_url_rumah123 = "https://www.rumah123.com/jual/rumah/" +"?page=" + str(i) # firstpage as page param
            header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'} # ini aku gapake perkenalan karena kirain gara2 tau ini "Tugus", di block
            request = Request(base_url_rumah123, headers = header)
            uClient = uReq(request)
            html_page = uClient.read()
            uClient.close()
            soup_page = soup(html_page, "lxml")

            print(f'Scraping {base_url_rumah123} ...\n')
            property_list = soup_page.findAll('div', class_='sc-fubCfw irPtrP')

            for prop in tqdm(property_list):
                try:
                    # listing property infos
                    property_featured_title = prop.find('div', class_='ui-organisms-card-r123-featured__title-info truncate relative ui-col')
                    property_title = property_featured_title.find('span', class_='ui-atomic-text ui-atomic-text--styling-heading-6 ui-atomic-text--typeface-primary ui-atomic-link--children').text

                    # bathroom, bedroom, and garage capacity
                    property_featured_bottom = prop.find('div', class_='ui-organisms-card-r123-featured__bottom-row flex flex-row flex-wrap ui-row flex-align-end flex-justify-between')
                    # array of max 3 elements (km kt g)
                    property_element = property_featured_bottom.findAll('span', 'ui-atomic-text ui-atomic-text--styling-default ui-atomic-text--typeface-primary ui-atomic-link--children') 
                    km = 0 if len(property_element) < 1 else int(property_element[0].text)
                    kt = 0 if len(property_element) < 2 else int(property_element[1].text)
                    garasi = 0 if len(property_element) < 3 else int(property_element[2].text)

                    # info regarding land and building area
                    property_area = prop.find('div', class_= 'flex ui-molecules-list__divider-line--horizontal flex-align-center flex-row flex-wrap relative ui-molecules-list')
                    # LT = Luas tanah, LB = Luas bangunan
                    try:
                        property_LT = property_area.text.split()[3]
                    except IndexError:
                        property_LT = 0
                    try:
                        property_LB = property_area.text.split()[7]
                    except IndexError:
                        property_LB = 0

                    if (km != 0 and kt != 0) and (property_LT != 0 and property_LB != 0):
                        # keep going

                        # agent info
                        agent = prop.find('h2').text

                        # phone info
                        if prop.find('div', class_='ui-atomic-card box-shadow-r123 ui-organisms-card-r123-featured ui-organisms-card-r123-featured--premier') is None:
                            phone_number = prop.find('div', class_='ui-atomic-card box-shadow-r123 ui-organisms-card-r123-featured')['phonenumber']
                        else:
                            phone_number = prop.find('div', class_='ui-atomic-card box-shadow-r123 ui-organisms-card-r123-featured ui-organisms-card-r123-featured--premier')['phonenumber']    
                        
                        phone_number = "+" + phone_number

                        # property type info
                        property_type = prop.find('div', class_='relative ui-col-12').text

                        # location info
                        property_location = property_featured_title.find('span', class_='ui-atomic-text ui-atomic-text--styling-default ui-atomic-text--typeface-primary ui-atomic-link--children').text

                        # price info
                        property_price_card = prop.find('div', class_='flex flex-align-baseline ui-organisms-card-r123-featured__price-info')
                        property_price = convert_harga_to_int(property_price_card.find('a', class_='ui-atomic-link ui-atomic-link--styling-underline-none').text)
                        
                        # creating object
                        coordinates = geocoder.geocode(str(property_location))
                        latitude = -2.4833826 if coordinates is None else coordinates.latitude
                        longitude = 117.8902853 if coordinates is None else coordinates.longitude
                        newHouse = HouseData(property_title, agent, property_location, latitude, longitude, phone_number, property_type, int(property_LT), int(property_LB), property_price, km, kt, garasi)
                        # newHouse.printObject()
                        listOfHouse.append(newHouse)
                        # wait 2 second
                        time.sleep(2)
                    else:
                        # we don't need bad data
                        pass
                
                except (AttributeError):
                    # listing other than house type
                    pass
                except Exception as e:
                    print(e)
            i+=1
            
        except Exception as e:
            print("\n")
            print('Error:', e)
            print("Error or Page finished")
            break
        
            
    print("\n")
    print(f"Scraping page from {firstpage} until {lastpage} has finished!")

## Create HouseData object list(s)

In [56]:
object1 = []
object2 = []

## Testing the web scraper function

This testing is done to make sure the web scraper isn't blocked by the website and is able to get data from the website

In [53]:
# testing to test if i'm blocked from the web
test = []
scrapeweb(test, 1,1)

Scraping https://www.rumah123.com/jual/rumah/?page=1 ...



100%|██████████| 20/20 [00:42<00:00,  2.13s/it]



Scraping page from 1 until 1 has finished!


In [54]:
# if len > 0, means we're not blocked
len(test)

18

## Scraping rumah123.com
The actual scraping. In order to not overload the website, I scraped the website with some intervals. This is done so that we don't get blocked because our suspicious requests made to the website

In [57]:
# scraping first 50 pages to obj1
scrapeweb(object1, 1, 50)

Scraping https://www.rumah123.com/jual/rumah/?page=1 ...



100%|██████████| 20/20 [00:40<00:00,  2.04s/it]


Scraping https://www.rumah123.com/jual/rumah/?page=2 ...



100%|██████████| 20/20 [00:43<00:00,  2.19s/it]


Scraping https://www.rumah123.com/jual/rumah/?page=3 ...



...

Scraping https://www.rumah123.com/jual/rumah/?page=49 ...



100%|██████████| 20/20 [00:45<00:00,  2.26s/it]


Scraping https://www.rumah123.com/jual/rumah/?page=50 ...



100%|██████████| 20/20 [00:45<00:00,  2.26s/it]



Scraping page from 1 until 50 has finished!


In [71]:
# checking if scraping is successful, length must be > 0
len(object1)

941

## Web Scrape 2nd Interval

In [65]:
scrapeweb(object2, 51,70)

Scraping https://www.rumah123.com/jual/rumah/?page=51 ...



100%|██████████| 20/20 [00:44<00:00,  2.24s/it]


Scraping https://www.rumah123.com/jual/rumah/?page=52 ...



100%|██████████| 20/20 [00:44<00:00,  2.24s/it]


Scraping https://www.rumah123.com/jual/rumah/?page=53 ...



...

Scraping https://www.rumah123.com/jual/rumah/?page=69 ...



100%|██████████| 20/20 [00:44<00:00,  2.23s/it]


Scraping https://www.rumah123.com/jual/rumah/?page=70 ...



100%|██████████| 20/20 [00:42<00:00,  2.12s/it]



Scraping page from 51 until 70 has finished!


In [40]:
len(object2)

378

## Web Scrape 3rd Interval

In [43]:
object3 = []
scrapewebalt(object3, 71, 90)

Scraping https://www.rumah123.com/jual/rumah/?page=71 ...



100%|██████████| 20/20 [01:45<00:00,  5.26s/it]


Scraping https://www.rumah123.com/jual/rumah/?page=72 ...



100%|██████████| 20/20 [01:44<00:00,  5.22s/it]


Scraping https://www.rumah123.com/jual/rumah/?page=73 ...



...

HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Jati+Asih%2C+Bekasi&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


100%|██████████| 20/20 [01:35<00:00,  4.79s/it]


Scraping https://www.rumah123.com/jual/rumah/?page=90 ...



100%|██████████| 20/20 [01:44<00:00,  5.23s/it]



Scraping page from 71 until 90 has finished!


In [66]:
len(object3)

386

## Web Scrape 4th (and final) Interval

In [67]:
object4 = []
scrapeweb(object4, 91, 100)

Scraping https://www.rumah123.com/jual/rumah/?page=91 ...



100%|██████████| 20/20 [00:46<00:00,  2.32s/it]


Scraping https://www.rumah123.com/jual/rumah/?page=92 ...



100%|██████████| 20/20 [00:40<00:00,  2.03s/it]


Scraping https://www.rumah123.com/jual/rumah/?page=93 ...



...

Scraping https://www.rumah123.com/jual/rumah/?page=99 ...



100%|██████████| 20/20 [00:45<00:00,  2.28s/it]


Scraping https://www.rumah123.com/jual/rumah/?page=100 ...



100%|██████████| 20/20 [00:43<00:00,  2.17s/it]



Scraping page from 91 until 100 has finished!


## Data Preprocessing

Because all the data are valid, we only need to concatenate the HouseData object list into a single list

In [68]:
# merging all object lists and checking overall length
listObj = object1+object2+object3+object4
len(listObj)

1905

## Output the list into JSON format

In order to convert a list of HouseData objects into a JSON format, we can use json.dump method provided by the JSON python library. We pass a list comprehenssion syntax [obj.__dict___ for obj in listObj] to generate a dictionary of all the objects inside the list, and output it to a JSON file 'Rumah123 Housing Data.json'

In [69]:
# convert list of objects to json format
with open('Rumah123 Housing Data.json', 'w') as f:
        json.dump([obj.__dict__ for obj in listObj], f, indent=4)